In [ ]:
# !pip install gymnasium
print("Installed gymnasium")

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import pandas as pd
import numpy as np

class MultiDataCenterEnvironment(gym.Env):
  metadata = {"render_modes": ["human"]}

  def _generate_workload(self):
    cpu_requirement = np.random.rand() * 100
    return np.float32(cpu_requirement)
  
  def _get_workload_datacentre(self, epsilon=0.5):
    r = np.random.rand()
    data_centre_to_send_work = self._workload_datacentre
    if r <= epsilon:
      data_centre_to_send_work = 1 - data_centre_to_send_work
    return data_centre_to_send_work

  def __init__(self, machines_data, datacentre_mapping, num_datacentres):
    super(MultiDataCenterEnvironment, self).__init__()
    
    assert len(machines_data) > 0
    assert len(machines_data[0]) > 0
    assert num_datacentres <= len(machines_data)

    self.num_datacentres = num_datacentres
    self.machines_data = machines_data
    self.datacentre_mapping = datacentre_mapping

    # Current time of the simulation
    self.current_time = 0
    # Current state of the simulation
    self._workload = self._generate_workload()
    self._workload_datacentre = 0
    self._machines_curr_state: list[np.float32] = [machine_data[0] for machine_data in self.machines_data]

    # (cpu1, cpu2, ..., cpu10, workload_requirement, data_center)
    self.observation_space = spaces.Dict(
      {
        "machines_curr_state": spaces.Tuple((spaces.Box(low=0, high=100, dtype=np.float32) for _ in range(len(machines_data)))),
        "workload": spaces.Box(low=0, high=100, dtype=np.float32),
        "workload_datacentre": spaces.Discrete(num_datacentres),
      }
    )
    
    # One action per machine and a no-op; choose to send the workload to a machine, or choose not to do anything
    self.action_space = spaces.Discrete(len(machines_data) + 1)

  def reset(self, seed=None, options=None):
    # We need the following line to seed self.np_random
    super().reset(seed=seed)

    # Current time of the simulation
    self.current_time = 0
    # Current state of the simulation
    self._workload = self._generate_workload()
    self._workload_datacentre = self._get_workload_datacentre()
    self._machines_curr_state: list[np.float32] = [np.array([machine_data[0]]) for machine_data in self.machines_data]

    observation = {
      "machines_curr_state": tuple(self._machines_curr_state),
      "workload": np.array([self._workload]),
      "workload_datacentre": self._workload_datacentre,
    }
    info = {} # Not sure what to use this for

    return observation, info
  
  # 10 machines, action A0 - A9, choose, no machine
  def step(self, action):
    observation = None
    reward = 0
    done = self.current_time >= (len(self.machines_data[0]) - 2)
    info = {} # Not sure what to use this for

    # Apply the action
    if action == 10:
      # Do nothing
      reward = 0
    else:
      machine_picked = action
      spare_capacity = 100 - self._machines_curr_state[machine_picked]
      # reward = min(100, int(spare_capacity / self._workload) * 100)
      if spare_capacity >= self._workload:
          reward = 100
      else:
        reward = int((spare_capacity / self._workload) * 100)

      # Half the reward if the datacentre is not where the workload originated
      if self._workload_datacentre != self.datacentre_mapping[machine_picked]:
        reward /= 2
    
    # Advance the time
    self.current_time += 1
    # Advance the state of the simulation
    self._workload = self._generate_workload()
    self._workload_datacentre = self._get_workload_datacentre()
    self._machines_curr_state: list[np.float32] = [np.array([machine_data[self.current_time]]) for machine_data in self.machines_data]

    observation = {
      "machines_curr_state": tuple(self._machines_curr_state),
      "workload": np.array([self._workload]),
      "workload_datacentre": self._workload_datacentre,
    }

    return observation, reward, done, False, info
    
print("Loaded environment")
    

In [ ]:
import random
from collections import deque

import torch
import torch.nn.functional as F
from gymnasium.core import Env
from torch import nn

class ReplayBuffer():
    def __init__(self, size:int, early_transitions_buffer_percentage:int = 0.1):
        """Replay buffer initialisation

        Args:
            size: maximum numbers of objects stored by replay buffer
        """
        self.size = size
        self.early_transitions_buffer_percentage = early_transitions_buffer_percentage
        self.early_transitions_buffer_size = int(early_transitions_buffer_percentage * size)
        # Reserve part of the buffer (default 10%) separate; do not overwrite any transitions here once filled
        self.early_transitions_buffer = deque([], self.early_transitions_buffer_size)
        self.rest_buffer_size = size - self.early_transitions_buffer_size
        self.rest_buffer = deque([], self.rest_buffer_size)
    
    def get_curr_size(self):
        return len(self.early_transitions_buffer) + len(self.rest_buffer)

    
    def push(self, transition):
        """Push an object to the replay buffer

        Args:
            transition: object to be stored in replay buffer. Can be of any type
        
        Returns:
            The current memory of the buffer (any iterable object e.g. list)
        """
        # Fill up the early_transitions_buffer until it is full
        if len(self.early_transitions_buffer) < self.early_transitions_buffer_size:
            self.early_transitions_buffer.append(transition)
        # If the early_transitions_buffer is full, we use the rest_buffer as usual
        else:
            self.rest_buffer.append(transition)
        return list(self.early_transitions_buffer) + list(self.rest_buffer)

    def sample(self, batch_size:int):
        """Get a random sample from the replay buffer
        
        Args:
            batch_size: size of sample

        Returns:
            iterable (e.g. list) with objects sampled from buffer without replacement
        """
        # Return a random sample from both buffers combined
        return random.sample(list(self.early_transitions_buffer) + list(self.rest_buffer), batch_size)


class DQN(nn.Module):
    def __init__(self, layer_sizes):
        """
        DQN initialisation

        Args:
            layer_sizes: list with size of each layer as elements
        """
        super(DQN, self).__init__()
        # torch.manual_seed(14597905165985114927) - This is a bad seed for a network of [4, 256, 2]
        self.layers = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes)-1)])
    
    def forward (self, x:torch.Tensor)->torch.Tensor:
        """Forward pass through the DQN

        Args:
            x: input to the DQN
        
        Returns:
            outputted value by the DQN
        """
        for layer in self.layers:
            x = F.leaky_relu(layer(x))
        return x

def greedy_action(dqn:DQN, state:torch.Tensor)->int:
    """Select action according to a given DQN
    
    Args:
        dqn: the DQN that selects the action
        state: state at which the action is chosen

    Returns:
        Greedy action according to DQN
    """
    return int(torch.argmax(dqn(state)))

def epsilon_greedy(epsilon:float, dqn:DQN, state:torch.Tensor)->int:
    """Sample an epsilon-greedy action according to a given DQN
    
    Args:
        epsilon: parameter for epsilon-greedy action selection
        dqn: the DQN that selects the action
        state: state at which the action is chosen
    
    Returns:
        Sampled epsilon-greedy action
    """
    q_values = dqn(state)
    num_actions = q_values.shape[0]
    greedy_act = int(torch.argmax(q_values))
    p = float(torch.rand(1))
    if p>epsilon:
        return greedy_act
    else:
        return random.randint(0,num_actions-1)

def update_target(target_dqn:DQN, policy_dqn:DQN):
    """Update target network parameters using policy network.
    Does not return anything but modifies the target network passed as parameter
    
    Args:
        target_dqn: target network to be modified in-place
        policy_dqn: the DQN that selects the action
    """

    target_dqn.load_state_dict(policy_dqn.state_dict())

def loss(policy_dqn:DQN, target_dqn:DQN,
         states:torch.Tensor, actions:torch.Tensor,
         rewards:torch.Tensor, next_states:torch.Tensor, dones:torch.Tensor, ddqn = False)->torch.Tensor:
    """Calculate Bellman error loss
    
    Args:
        policy_dqn: policy DQN
        target_dqn: target DQN
        states: batched state tensor
        actions: batched action tensor
        rewards: batched rewards tensor
        next_states: batched next states tensor
        dones: batched Boolean tensor, True when episode terminates
    
    Returns:
        Float scalar tensor with loss value
    """
    if ddqn:
        # DDQN Start:
        # torch.Size([BATCH_SIZE, 1])
        policy_dqn_best_action_batch = torch.argmax(policy_dqn(next_states), dim=1).reshape(-1, 1) # select best actions based on policy dqn
        # torch.Size([BATCH_SIZE, 2])
        target_dqn_next_states = target_dqn(next_states)
        # torch.Size([BATCH_SIZE, 1])
        selected_action_batch = target_dqn_next_states.gather(dim=1, index=policy_dqn_best_action_batch) # use actions on the target dqn

        # dones shape: torch.Size([BATCH_SIZE, 1]), reshaped(-1): torch.Size.([BATCH_SIZE])
        # rewards shape: torch.Size([BATCH_SIZE, 1]), reshaped(-1): torch.Size([BATCH_SIZE])
        # selected_action_batch reshaped(-1): torch.Size([BATCH_SIZE])
        # bellman_targets shape: torch.Size([BATCH_SIZE])
        bellman_targets = (~dones).reshape(-1)*(selected_action_batch).reshape(-1) + rewards.reshape(-1)
        # DDQN End
    else:
        bellman_targets = (~dones).reshape(-1)*(target_dqn(next_states)).max(1).values + rewards.reshape(-1)

    q_values = policy_dqn(states).gather(1, actions).reshape(-1)
    return ((q_values - bellman_targets)**2).mean()

print("Loaded utils")

In [ ]:
import os
import pandas as pd
import numpy as np

# Set up environment

machines_data = []

print ("Opening files machine data files:")
path_to_machine_data = os.path.join(os.getcwd(), "..", "input", "machine-data")
for filename in os.listdir(path_to_machine_data):
  df = pd.read_csv(os.path.join(path_to_machine_data, filename))
  machine_name = os.path.splitext(filename)[0]
#   print(machine_name)
#   print(df.head())
  machine_data = [np.int32(row[1]) for row in df.values.tolist()]
  machines_data.append(machine_data)

num_datacentres = 2
datacentre_mapping = {
  0: 0, 1: 0, 2: 0, 3: 0, 4: 0,
  5: 1, 6: 1, 7: 1, 8: 1, 9: 1,
}

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math
import numpy as np

import gymnasium as gym
import matplotlib.pyplot as plt
print("Imports")

In [ ]:
# Test main code
# env = gym.make('custom_environments/MultiDataCenterEnvironment', 
#                     machines_data=machines_data, 
#                     datacentre_mapping=datacentre_mapping, 
#                     num_datacentres=num_datacentres
#                   )
env = MultiDataCenterEnvironment(
                    machines_data=machines_data, 
                    datacentre_mapping=datacentre_mapping, 
                    num_datacentres=num_datacentres
                  )


terminated = False
done = False
env.reset()
time = 0
random_action_ret = 0
while not (done or terminated):
  random_action = np.random.randint(0, 11)
  observation, reward, done, terminated, info = env.step(random_action)
  flattened_observation = np.concatenate((
                                            (np.array(observation["machines_curr_state"]) / 100).ravel(), 
                                            (observation["workload"] / 100).ravel(), 
                                            np.array([0. if (observation["workload_datacentre"] == 0) else 1.])
                                        ))
#   flattened_normalised_observation = (flattened_observation-np.min(flattened_observation))/(np.max(flattened_observation)-np.min(flattened_observation))
  print(time)
  print(flattened_observation)
  print("Reward: ", reward)
  random_action_ret += reward
  time += 1

print ("Random Action Return:", random_action_ret)

In [ ]:
# Test main code
# env = gym.make('custom_environments/MultiDataCenterEnvironment', 
#                     machines_data=machines_data, 
#                     datacentre_mapping=datacentre_mapping, 
#                     num_datacentres=num_datacentres
#                   )
env = MultiDataCenterEnvironment(
                    machines_data=machines_data, 
                    datacentre_mapping=datacentre_mapping, 
                    num_datacentres=num_datacentres
                  )


terminated = False
done = False
env.reset()
time = 0
greedy_action_ret = 0
while not (done or terminated):
  greedy_action = 10
  most_spare_capacity = 0
  for i, machine_data in enumerate(machines_data):
    spare_capacity = 100 - machine_data[time]
    if spare_capacity > most_spare_capacity:
        most_spare_capacity = spare_capacity
        greedy_action = i
    

  observation, reward, done, terminated, info = env.step(greedy_action)
  flattened_observation = np.concatenate((
                                            (np.array(observation["machines_curr_state"]) / 100).ravel(), 
                                            (observation["workload"] / 100).ravel(), 
                                            np.array([0. if (observation["workload_datacentre"] == 0) else 1.])
                                        ))
#   flattened_normalised_observation = (flattened_observation-np.min(flattened_observation))/(np.max(flattened_observation)-np.min(flattened_observation))
#   print(time)
#   print(flattened_observation)
#   print("Reward: ", reward)
  greedy_action_ret += reward
  time += 1

print ("Greedy Action Return:", greedy_action_ret)

In [ ]:
def run_experiment(
        NUM_RUNS = 1, 
        EPSILON_DECAY=0.97, 
        HIDDEN_LAYER_SIZE=512, 
        LEARNING_RATE = 0.01, 
        REPLAY_BUFFER_SIZE=30_000,
        NUM_EPISODES=300,
        REPLAY_BUFFER_MINIBATCH_SIZE=64,
        NETWORK_SYNC_FREQUENCY=50,
        ddqn=False
    ):
    print("DDQN:", ddqn)
    EPSILON_START = 1
    EPSILON_MIN = 0.001

    # Hyperparameters
    INPUT_LAYER_SIZE = 12 # 10 machines + workload + workload datacentre
    OUTPUT_LAYER_SIZE = 11 # 10 machines (1 action per machine) + noop action
    POLICY_NET_LAYER_SIZES = [INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE]
    TARGET_NET_LAYER_SIZES = [INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE]
    LEARNING_RATE = LEARNING_RATE
    REPLAY_BUFFER_SIZE = REPLAY_BUFFER_SIZE
    REPLAY_BUFFER_MINIBATCH_SIZE = REPLAY_BUFFER_MINIBATCH_SIZE
    NETWORK_SYNC_FREQUENCY = NETWORK_SYNC_FREQUENCY
    
    env = MultiDataCenterEnvironment(
                    machines_data=machines_data, 
                    datacentre_mapping=datacentre_mapping, 
                    num_datacentres=num_datacentres
                  )
    runs_results = []
    
    for run in range(NUM_RUNS):
        print(f"Starting run {run+1} of {NUM_RUNS}")
        policy_net = DQN(POLICY_NET_LAYER_SIZES)
        target_net = DQN(TARGET_NET_LAYER_SIZES)
        EPSILON = EPSILON_START
        update_target(target_net, policy_net)
        target_net.eval()

        optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
        memory = ReplayBuffer(REPLAY_BUFFER_SIZE)

        steps_done = 0

        episode_returns = []

        for i_episode in range(NUM_EPISODES):
            print("episode ", i_episode+1, "/", NUM_EPISODES)

            observation, info = env.reset()
            flattened_observation = np.concatenate((
                                            (np.array(observation["machines_curr_state"]) / 100).ravel(), 
                                            (observation["workload"] / 100).ravel(), 
                                            np.array([0. if (observation["workload_datacentre"] == 0) else 1.]) # one hot encode the datacentres
                                        ))
            state = torch.tensor(flattened_observation).float()

            done = False
            terminated = False
            ret = 0
            while not (done or terminated):

                # Select and perform an action
                action = epsilon_greedy(EPSILON, policy_net, state)

                observation, reward, done, terminated, info = env.step(action)
                ret += reward
                flattened_observation = np.concatenate((
                                            (np.array(observation["machines_curr_state"]) / 100).ravel(), 
                                            (observation["workload"] / 100).ravel(), 
                                            np.array([0. if (observation["workload_datacentre"] == 0) else 1.]) # one hot encode the datacentres
                                        ))
                reward = torch.tensor([reward])
                action = torch.tensor([action])
                next_state = torch.tensor(flattened_observation).reshape(-1).float()

                memory.push([state, action, next_state, reward, torch.tensor([done])])

                # Move to the next state
                state = next_state

                steps_done += 1

                # Perform one step of the optimization (on the policy network)
                if not memory.get_curr_size() < REPLAY_BUFFER_MINIBATCH_SIZE:
                    transitions = memory.sample(REPLAY_BUFFER_MINIBATCH_SIZE)
                    state_batch, action_batch, nextstate_batch, reward_batch, dones = (torch.stack(x) for x in zip(*transitions))
                    # Compute loss
                    mse_loss = loss(policy_net, target_net, state_batch, action_batch, reward_batch, nextstate_batch, dones, ddqn=ddqn)
                    # Optimize the model
                    optimizer.zero_grad()
                    mse_loss.backward()
                    optimizer.step()

                if steps_done % NETWORK_SYNC_FREQUENCY == 0:
                    update_target(target_net, policy_net)

                if done or terminated:
                    episode_returns.append(ret)

            # Update the target network, copying all weights and biases in DQN
            # if i_episode % NETWORK_SYNC_FREQUENCY == 0: 
            #     update_target(target_net, policy_net)

            # Update epsilon
            EPSILON = max(EPSILON * EPSILON_DECAY, EPSILON_MIN)
        runs_results.append(episode_returns)
    print('Complete')
    return policy_net, runs_results


# def run_trained_policy(policy_net, NUM_RUNS = 5, NUM_EPISODES=300):
#     env = gym.make('custom_environments/MultiDataCenterEnvironment', 
#                     machines_data=machines_data, 
#                     datacentre_mapping=datacentre_mapping, 
#                     num_datacentres=num_datacentres
#                   )
#     runs_results = []

#     for run in range(NUM_RUNS):
#         print(f"Starting run {run+1} of {NUM_RUNS}")
#         steps_done = 0

#         episode_durations = []

#         for i_episode in range(NUM_EPISODES):
#             if (i_episode+1) % 50 == 0:
#                 print("episode ", i_episode+1, "/", NUM_EPISODES)

#             observation, info = env.reset()
#             state = torch.tensor(observation).float()

#             done = False
#             terminated = False
#             t = 0
#             while not (done or terminated):
#                 # Select and perform an action
#                 action = epsilon_greedy(0, policy_net, state)

#                 observation, reward, done, terminated, info = env.step(action)
#                 next_state = torch.tensor(observation).reshape(-1).float()

#                 # Move to the next state
#                 state = next_state

#                 steps_done += 1

#                 if done or terminated:
#                     episode_durations.append(t + 1)
#                 t += 1

#         runs_results.append(episode_durations)
#     print('Complete')
#     return runs_results

print ("Defined run_experiment")

In [ ]:
def plot_graph(ddqn_runs_results, NUM_EPISODES, filename):
    ddqn_results = torch.tensor(ddqn_runs_results)
    ddqn_means = ddqn_results.float().mean(0)
    ddqn_stds = ddqn_results.float().std(0)

    plt.plot(torch.arange(NUM_EPISODES), ddqn_means, label="DDQN Mean", color='orange')
    plt.ylabel("return")
    plt.xlabel("episode")

    plt.fill_between(np.arange(NUM_EPISODES), ddqn_means + ddqn_stds, ddqn_means - ddqn_stds, alpha=0.3, label=f"DDQN Mean ± Std", color='orange')
    plt.axhline(y=random_action_ret, color='r', linestyle='--')
    plt.legend(loc='lower right')
    plt.savefig(f'/kaggle/working/{filename}.png')
    plt.show()

print("Defined plot_graph")

In [ ]:
# Hyperparameter search
# hidden_layer_sizes = [128, 256, 512]
# learning_rates = [0.0001, 0.0005, 0.001, 0.005, 0.01]
# epsilon_decays = [0.999, 0.995, 0.99, 0.98, 0.97, 0.96]
# minibatch_sizes = [32, 64, 128, 256]
# network_sync_freqs = [50, 100, 200, 400]

hidden_layer_sizes = [512]
learning_rates = [0.002]
epsilon_decays = [0.98]
minibatch_sizes = [256]
network_sync_freqs = [5000, 7500, 10_000, 15_000, 20_000]

num_episodes = 300

for hidden_layer_size in hidden_layer_sizes:
    for learning_rate in learning_rates:
        for epsilon_decay in epsilon_decays:
            for minibatch_size in minibatch_sizes:
                for network_sync_freq in network_sync_freqs:
                    print ("hidden layer size:", hidden_layer_size, "LR:", learning_rate, "eps decay:", epsilon_decay, "minib size:", minibatch_size, "sync freq:", network_sync_freq)
                    ddqn_policy_net, ddqn_runs_results = run_experiment(
                                                                NUM_RUNS = 1,
                                                                EPSILON_DECAY=epsilon_decay, 
                                                                HIDDEN_LAYER_SIZE=hidden_layer_size,
                                                                LEARNING_RATE = learning_rate,
                                                                REPLAY_BUFFER_SIZE=30_000,
                                                                NUM_EPISODES=num_episodes,
                                                                REPLAY_BUFFER_MINIBATCH_SIZE=minibatch_size,
                                                                NETWORK_SYNC_FREQUENCY=network_sync_freq,
                                                                ddqn=True
                                                            )
                    filename = f"HLS_{hidden_layer_size}_lr_{learning_rate}_epsdec_{epsilon_decay}_MBS_{minibatch_size}_syncfreq_{network_sync_freq}"
                    plot_graph(ddqn_runs_results, num_episodes, filename)